In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix

from tensorflow.python.keras.models import load_model

from utility import clean_data, plot_images
import math

D:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

In [2]:
classes = ['angry','disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']

In [3]:
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap='Blues'):
    if not title:
        if normalize:
            title = 'Normalized Confusion Matrix'
        else:
            title = 'Confusion Matrix, without Normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    # classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax


In [4]:
train_data = clean_data('data/train.csv')
train = train_data.feature.reshape((-1, 48, 48, 1))/255
train_x = train[:-2000]
train_label = train_data.label[:-2000]
train_onehot = train_data.onehot[:-2000]
test_x = train[-2000:]
test_label = train_data.label[-2000:]
test_onehot = train_data.onehot[-2000:]

In [5]:
cnn = load_model('cnn.h5')
dnn = load_model('dnn.h5')

OSError: Unable to open file (unable to open file: name = 'cnn2.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
cnn_predict = cnn.predict(test_x)
cnn_cls = np.argmax(cnn_predict, axis=1)

In [ ]:
dnn_predict = dnn.predict(test_x)
dnn_cls = np.argmax(dnn_predict, axis=1)

In [ ]:
plot_confusion_matrix(test_label, cnn_cls, classes)

In [ ]:
plot_confusion_matrix(test_label, dnn_cls, classes)

In [ ]:
true_cls = pd.Series(test_label, name='true_cls')[test_label!=cnn_cls]
wrong_cls = pd.Series(cnn_cls, name='wrong_cls')[test_label!=cnn_cls]

In [ ]:
wrong = pd.concat([true_cls, wrong_cls], axis = 1)

In [ ]:
sad_neu = wrong[wrong['true_cls']==4][wrong['wrong_cls']==6].index.tolist()

In [ ]:
sample = sad_neu[0:9]

In [ ]:
plot_images(test_x[sample], test_label[sample], cnn_cls[sample])

In [ ]:
from vis.visualization import visualize_saliency, overlay
from vis.utils import utils
from keras import activations
from keras.models import load_model



In [ ]:
cnn = load_model('cnn.h5')

In [ ]:
# Swap softmax with linear
cnn.layers[-1].activation = activations.linear
cnn = utils.apply_modifications(cnn)

grads = visualize_saliency(cnn, -1, filter_indices=test_label[0], seed_input=test_x[0], backprop_modifier='guided')
# Plot with 'jet' colormap to visualize as a heatmap.
plt.imshow(grads, cmap='jet')

In [ ]:
for idx in range(6):    
    

    f, ax = plt.subplots(1, 4)
    ax[0].imshow(test_x[idx].reshape((48,48)), cmap='gray')
    
    for i, modifier in enumerate([None, 'guided', 'relu']):
        grads = visualize_saliency(cnn, -1, filter_indices=test_label[idx], 
                                   seed_input=test_x[idx], backprop_modifier=modifier)
        if modifier is None:
            modifier = 'vanilla'
        ax[i+1].set_title(modifier)    
        ax[i+1].imshow(grads, cmap='jet')

### 分析filter

In [ ]:
cnn.summary()

In [ ]:
layer_input = cnn.layers[0]
layer_conv1 = cnn.layers[1]
layer_conv2 = cnn.layers[5]
layer_conv3 = cnn.layers[9]
conv_layers = [layer_conv1, layer_conv2, layer_conv3]

In [ ]:
layer_conv1.get_weights()[0].shape

In [ ]:
def plot_conv_weights(weights, input_channel=0):
    # Get the lowest and highest values for the weights.
    # This is used to correct the colour intensity across
    # the images so they can be compared with each other.
    w_min = np.min(weights)
    w_max = np.max(weights)

    # Number of filters used in the conv. layer.
    num_filters = weights.shape[3]

    # Number of grids to plot.
    # Rounded-up, square-root of the number of filters.
    num_grids = math.ceil(math.sqrt(num_filters))
    
    # Create figure with a grid of sub-plots.
    fig, axes = plt.subplots(8, 8)

    # Plot all the filter-weights.
    for i, ax in enumerate(axes.flat):
        # Only plot the valid filter-weights.
        if i<num_filters:
            # Get the weights for the i'th filter of the input channel.
            # See new_conv_layer() for details on the format
            # of this 4-dim tensor.
            img = weights[:, :, input_channel, i]

            # Plot image.
            ax.imshow(img, vmin=w_min, vmax=w_max,
                      interpolation='nearest', cmap='seismic')
        
        # Remove ticks from the plot.
        ax.set_xticks([])
        ax.set_yticks([])
    
    # Ensure the plot is shown correctly with multiple plots
    # in a single Notebook cell.
    plt.show()

In [ ]:
for conv_layer in conv_layers:
    plot_conv_weights(conv_layer.get_weights()[0]) 

### 分析layer

In [ ]:
def plot_conv_output(values):
    # Number of filters used in the conv. layer.
    num_filters = values.shape[3]

    # Number of grids to plot.
    # Rounded-up, square-root of the number of filters.
    num_grids = math.ceil(math.sqrt(num_filters))
    
    # Create figure with a grid of sub-plots.
    fig, axes = plt.subplots(8, 8)

    # Plot the output images of all the filters.
    for i, ax in enumerate(axes.flat):
        # Only plot the images for valid filters.
        if i<num_filters:
            # Get the output image of using the i'th filter.
            img = values[0, :, :, i]

            # Plot image.
            ax.imshow(img, interpolation='nearest', cmap='Blues')
        
        # Remove ticks from the plot.
        ax.set_xticks([])
        ax.set_yticks([])
    
    # Ensure the plot is shown correctly with multiple plots
    # in a single Notebook cell.
    plt.show()

In [ ]:
from keras import backend as K

In [ ]:
image = test_x[0]

In [ ]:
plt.imshow(image.reshape(48,48), cmap='gray')

In [ ]:
output_conv1 = K.function(inputs=[layer_input.input],
                          outputs=[layer_conv1.output])
output_conv2 = K.function(inputs=[layer_input.input],
                          outputs=[layer_conv2.output])
output_conv3 = K.function(inputs=[layer_input.input],
                          outputs=[layer_conv3.output])

In [ ]:
layer_output1 = output_conv1([[image]])[0]
layer_output2 = output_conv2([[image]])[0]
layer_output3 = output_conv3([[image]])[0]

In [ ]:
plot_conv_output(values=layer_output1)

In [ ]:
plot_conv_output(values=layer_output2)

In [ ]:
plot_conv_output(values=layer_output3)

In [ ]:
# build a loss function that maximizes the activation
# of the nth filter of the layer considered
layer_output = layer_conv1.output
filter_index = 0
loss = K.mean(layer_output[:, :, :, filter_index])

# compute the gradient of the input picture wrt this loss
grads = K.gradients(loss, layer_input.input)[0]

# normalization trick: we normalize the gradient
grads /= (K.sqrt(K.mean(K.square(grads))) + 1e-5)

# this function returns the loss and grads given the input picture
iterate = K.function([layer_input.input], [loss, grads])

In [ ]:
# we start from a gray image with some noise
input_img_data = np.random.random((1, 48, 48, 1)) * 20 + 128.
# run gradient ascent for 20 steps
for i in range(500):
    loss_value, grads_value = iterate([input_img_data])
    input_img_data += grads_value * 0.1

In [ ]:
plt.imshow(input_img_data.reshape(48,48), cmap='gray')

In [ ]:
def _filter(layer_conv):
    result = []
    layer_output = layer_conv.output
    for filter_index in range(64):
        loss = K.mean(layer_output[:, :, :, filter_index])
        # compute the gradient of the input picture wrt this loss
        grads = K.gradients(loss, layer_input.input)[0]

        # normalization trick: we normalize the gradient
        grads /= (K.sqrt(K.mean(K.square(grads))) + 1e-5)

        # this function returns the loss and grads given the input picture
        iterate = K.function([layer_input.input], [loss, grads])
        input_img_data = np.random.random((1, 48, 48, 1)) * 20 + 128.
        # run gradient ascent for 20 steps
        for i in range(500):
            loss_value, grads_value = iterate([input_img_data])
            input_img_data += grads_value * 0.1
        result.append(input_img_data.reshape(48,48))
        
    return result

In [ ]:
def plot_filter(layer_conv):
     # Create figure with a grid of sub-plots.
    fig, axes = plt.subplots(8, 8)
    img = _filter(layer_conv)
    # Plot the output images of all the filters.
    for i, ax in enumerate(axes.flat):

        ax.imshow(img[i], interpolation='nearest', cmap='Blues')
        
        # Remove ticks from the plot.
        ax.set_xticks([])
        ax.set_yticks([])
    
    # Ensure the plot is shown correctly with multiple plots
    # in a single Notebook cell.
    plt.show()

In [ ]:
plot_filter(layer_conv1)

In [ ]:
plot_filter(layer_conv2)

In [ ]:
plot_filter(layer_conv3)